In [1]:
import spacy
from spacy import displacy

text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

nlp = spacy.load('en')
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

In [2]:
colors = {'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)'}
options = {'ents': ['ORG'], 'colors': colors}
displacy.render(doc, style='ent', jupyter=True, options=options)

In [3]:
doc.ents

(Google, , Apple, iPhones, Amazon, Alexa, Echo, Dot, )

In [4]:
doc[2:5]

is starting from

In [24]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
doc.to_array([LOWER, POS, ENT_TYPE])

array([[14560795576765492085,                   88,                    0],
       [ 1988622737398120358,                   95,                  381],
       [ 3411606890003347522,                   99,                    0],
       [17219656379376048755,                   99,                    0],
       [ 7831658034963690409,                   84,                    0],
       [ 9368086581607646285,                   85,                    0],
       [12646065887601541794,                   96,                    0],
       [ 7425985699627899538,                   89,                    0],
       [ 6905553075311563409,                   91,                    0],
       [ 7136294487848412980,                   99,                    0],
       [11901859001352538922,                   89,                    0],
       [ 7199752172782760331,                   83,                    0],
       [ 4198392880578328572,                   91,                    0],
       [  962983613142996

In [6]:
np_array = doc.to_array([ENT_TYPE])

In [7]:
np_array.shape

(50,)

In [8]:
np_array[6] = 384

In [9]:
np_array = np_array.reshape((len(np_array),1))

In [10]:
np_array.shape

(50, 1)

In [12]:
from spacy.tokens import Doc
doc2 = Doc(doc.vocab, words=[t.text for t in doc])
doc2.from_array([ENT_TYPE], np_array)

But Google is starting from behind . The company made a late push 
 into hardware , and Apple ’s Siri , available on iPhones , and Amazon ’s Alexa 
 software , which runs on its Echo and Dot devices , have clear leads in 
 consumer adoption . 

In [15]:
doc2.to_array([ENT_TYPE])

array([  0, 381,   0,   0,   0,   0, 384,   0,   0,   0,   0,   0,   0,
       382,   0,   0,   0,   0, 381,   0,   0,   0,   0,   0, 384,   0,
         0, 381,   0, 381, 381,   0,   0,   0,   0,   0,   0, 382,   0,
       381,   0,   0,   0,   0,   0,   0, 382,   0,   0,   0],
      dtype=uint64)

In [20]:
doc2.ents = doc.ents

In [22]:
displacy.render(doc2, style='ent', jupyter=True)

In [37]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
doc = nlp("Hello world!")
np_array = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])
doc2 = Doc(doc.vocab, words=[t.text for t in doc])
doc2.from_array([LOWER, POS, ENT_TYPE, IS_ALPHA], np_array)
assert doc[0].pos_ == doc2[0].pos_

In [38]:
doc2

Hello world ! 

In [1]:
from __future__ import unicode_literals, print_function

import plac
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc, Span, Token


@plac.annotations(
    text=("Text to process", "positional", None, str),
    companies=("Names of technology companies", "positional", None, str))

def main(text="Alphabet Inc. is the company behind Google.", *companies):
    # For simplicity, we start off with only the blank English Language class
    # and no model or pre-defined pipeline loaded.
    nlp = English()
    if not companies:  # set default companies if none are set via args
        companies = ['Alphabet Inc.', 'Google', 'Netflix', 'Apple']  # etc.
    component = TechCompanyRecognizer(nlp, companies)  # initialise component
    nlp.add_pipe(component, last=True)  # add last to the pipeline

    doc = nlp(text)
    print('Pipeline', nlp.pipe_names)  # pipeline contains component name
    print('Tokens', [t.text for t in doc])  # company names from the list are merged
    print('Doc has_tech_org', doc._.has_tech_org)  # Doc contains tech orgs
    print('Token 0 is_tech_org', doc[0]._.is_tech_org)  # "Alphabet Inc." is a tech org
    print('Token 1 is_tech_org', doc[1]._.is_tech_org)  # "is" is not
    print('Entities', [(e.text, e.label_) for e in doc.ents])  # all orgs are entities


class TechCompanyRecognizer(object):
    """Example of a spaCy v2.0 pipeline component that sets entity annotations
    based on list of single or multiple-word company names. Companies are
    labelled as ORG and their spans are merged into one token. Additionally,
    ._.has_tech_org and ._.is_tech_org is set on the Doc/Span and Token
    respectively."""
    name = 'tech_companies'  # component name, will show up in the pipeline

    def __init__(self, nlp, companies=tuple(), label='ORG'):
        """Initialise the pipeline component. The shared nlp instance is used
        to initialise the matcher with the shared vocab, get the label ID and
        generate Doc objects as phrase match patterns.
        """
        self.label = nlp.vocab.strings[label]  # get entity label ID

        # Set up the PhraseMatcher – it can now take Doc objects as patterns,
        # so even if the list of companies is long, it's very efficient
        patterns = [nlp(org) for org in companies]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add('TECH_ORGS', None, *patterns)

        # Register attribute on the Token. We'll be overwriting this based on
        # the matches, so we're only setting a default value, not a getter.
        Token.set_extension('is_tech_org', default=False)

        # Register attributes on Doc and Span via a getter that checks if one of
        # the contained tokens is set to is_tech_org == True.
        Doc.set_extension('has_tech_org', getter=self.has_tech_org)
        Span.set_extension('has_tech_org', getter=self.has_tech_org)

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        matches = self.matcher(doc)
        spans = []  # keep the spans for later so we can merge them afterwards
        for _, start, end in matches:
            # Generate Span representing the entity & set label
            entity = Span(doc, start, end, label=self.label)
            spans.append(entity)
            # Set custom attribute on each token of the entity
            for token in entity:
                token._.set('is_tech_org', True)
            # Overwrite doc.ents and add entity – be careful not to replace!
            doc.ents = list(doc.ents) + [entity]
        for span in spans:
            # Iterate over all spans and merge them into one token. This is done
            # after setting the entities – otherwise, it would cause mismatched
            # indices!
            span.merge()
        return doc  # don't forget to return the Doc!

    def has_tech_org(self, tokens):
        """Getter for Doc and Span attributes. Returns True if one of the tokens
        is a tech org. Since the getter is only called when we access the
        attribute, we can refer to the Token's 'is_tech_org' attribute here,
        which is already set in the processing step."""
        return any([t._.get('is_tech_org') for t in tokens])


In [2]:
plac.call(main)

Pipeline ['tech_companies']
Tokens ['/home', '/', 'mlelarge/.local', '/', 'share', '/', 'jupyter', '/', 'runtime', '/', 'kernel-3b889537-eaa5', '-', '4e7b-8563-c410d12773fe.json']
Doc has_tech_org False
Token 0 is_tech_org False
Token 1 is_tech_org False
Entities []


In [11]:
text

'But Google is starting from behind. The company made a late push\ninto hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa\nsoftware, which runs on its Echo and Dot devices, have clear leads in\nconsumer adoption.'